In [17]:
from pymongo import MongoClient

# MongoDB에 연결
client = MongoClient('mongodb://localhost:27017/')  # mongo-router의 포트로 연결
db = client['cycle']  # cycle 데이터베이스 선택

# 컬렉션 목록 가져오기
collections = db.list_collection_names()

# 결과 출력
print("데이터베이스 'cycle'의 컬렉션 목록:")
for collection in collections:
    print(collection)

# 특정 컬렉션의 데이터 확인
broken_history = db['broken_history']
station_master = db['station_master']
rental_shop_info = db['rental_shop_info']
rental_info = db['rental_info']
document_count1 = broken_history.count_documents({})
document_count2 = station_master.count_documents({})
document_count3 = rental_shop_info.count_documents({})
document_count4 = rental_info.count_documents({})
print(f"'broken_history' 컬렉션에 있는 문서 수: {document_count1}")
print(f"'station_master' 컬렉션에 있는 문서 수: {document_count2}")
print(f"'rental_shop_info' 컬렉션에 있는 문서 수: {document_count3}")
print(f"'rental_info' 컬렉션에 있는 문서 수: {document_count4}")

데이터베이스 'cycle'의 컬렉션 목록:
station_master
broken_history
rental_shop_info
rental_info
'broken_history' 컬렉션에 있는 문서 수: 30513
'station_master' 컬렉션에 있는 문서 수: 967
'rental_shop_info' 컬렉션에 있는 문서 수: 3286
'rental_info' 컬렉션에 있는 문서 수: 46388233


In [18]:
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure

# MongoDB 클러스터의 Mongo Router에 연결
mongo_router_uri = "mongodb://localhost:27017"
client = MongoClient(mongo_router_uri)

def check_connection(client):
    try:
        # 클러스터 상태 확인
        server_info = client.server_info()
        print("Connected to MongoDB server version:", server_info["version"])
        print("Cluster status: Connected")
    except ConnectionFailure as e:
        print("Could not connect to MongoDB:", e)

def check_shard_status(client):
    # 샤드 상태 확인
    shards = client.admin.command('listShards')
    print("Shards in the cluster:")
    for shard in shards['shards']:
        print(f"Shard: {shard['_id']}, Host: {shard['host']}")

def check_database_collections(client, db_name):
    db = client[db_name]
    collections = db.list_collection_names()
    print(f"Collections in database '{db_name}':")
    for collection in collections:
        print(f" - {collection}")

# 연결 상태 확인
check_connection(client)

# 샤드 상태 확인
check_shard_status(client)

# 특정 데이터베이스의 컬렉션 상태 확인 (예: cycle 데이터베이스)
check_database_collections(client, 'cycle')

Connected to MongoDB server version: 4.4.29
Cluster status: Connected
Shards in the cluster:
Shard: shard1ReplSet, Host: shard1ReplSet/mongo-shard1:27022
Shard: shard2ReplSet, Host: shard2ReplSet/mongo-shard2:27023
Collections in database 'cycle':
 - station_master
 - broken_history
 - rental_shop_info
 - rental_info


In [21]:
# 자전거 고장 신고 내역에서 가장 많이 발생한 고장 유형 조회
from pymongo import MongoClient

client = MongoClient('mongodb://127.0.0.1:27017')
db = client['cycle']
collection = db['broken_history']

pipeline = [
    {"$group": {"_id": "$content", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}},
    {"$limit": 5}
]

try:
    result = list(collection.aggregate(pipeline))
    for doc in result:
        print(doc)
except Exception as e:
    print(f"An error occurred: {e}")

{'_id': '기타 ', 'count': 10190}
{'_id': '체인', 'count': 5678}
{'_id': '타이어 ', 'count': 5304}
{'_id': '안장', 'count': 5001}
{'_id': '페달', 'count': 2734}


In [22]:
# 자전거 대여소별 대여 횟수 조회
collection = db['rental_info']

pipeline = [
    {"$group": {"_id": "$rental_place", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}},
    {"$limit": 5}
]

result = list(collection.aggregate(pipeline))
for doc in result:
    print(doc)

{'_id': '마곡나루역 2번 출구', 'count': 187606}
{'_id': '한강공원 망원나들목', 'count': 142837}
{'_id': '뚝섬유원지역 1번출구 앞', 'count': 139984}
{'_id': '여의나루역 1번출구 앞', 'count': 119607}
{'_id': '마곡나루역 5번출구 뒤편', 'count': 118033}


In [23]:
# 특정 대여소에서의 평균 대여 시간
collection = db['rental_info']

pipeline = [
    {"$match": {"rental_place": "번동사거리"}},
    {"$group": {"_id": "$rental_place", "average_use_time": {"$avg": "$use_time_minute"}}}
]

result = list(collection.aggregate(pipeline))
for doc in result:
    print(doc)

{'_id': '번동사거리', 'average_use_time': 23.95011100953376}


In [24]:
# 자전거 고장 신고 내역과 대여 내역 조인 (Aggregation with $lookup)
collection = db['broken_history']

pipeline = [
    {
        "$lookup": {
            "from": "rental_info",
            "localField": "cycle_num",
            "foreignField": "cycle_num",
            "as": "rental_info"
        }
    },
    {"$unwind": "$rental_info"},
    {"$project": {
        "cycle_num": 1,
        "created_at": 1,
        "content": 1,
        "rental_info.rental_date": 1,
        "rental_info.rental_place": 1,
        "rental_info.return_date": 1,
        "rental_info.return_place": 1
    }},
    {"$limit": 5}
]

result = list(collection.aggregate(pipeline))
for doc in result:
    print(doc)

{'_id': ObjectId('66534e5f0b54fa89dc401c63'), 'cycle_num': 'SPB-62819', 'created_at': datetime.datetime(2023, 1, 1, 0, 1), 'content': '안장', 'rental_info': {'rental_date': datetime.datetime(2023, 1, 1, 22, 25, 49), 'rental_place': '동대문역사문화공원역 1번출구 뒤편', 'return_date': datetime.datetime(2023, 1, 1, 22, 35, 36), 'return_place': '왕십리교회옆'}}
{'_id': ObjectId('66534e5f0b54fa89dc401c63'), 'cycle_num': 'SPB-62819', 'created_at': datetime.datetime(2023, 1, 1, 0, 1), 'content': '안장', 'rental_info': {'rental_date': datetime.datetime(2023, 1, 3, 9, 58, 6), 'rental_place': '왕십리교회옆', 'return_date': datetime.datetime(2023, 1, 3, 10, 9, 9), 'return_place': '동묘앞역 1번출구 뒤'}}
{'_id': ObjectId('66534e5f0b54fa89dc401c63'), 'cycle_num': 'SPB-62819', 'created_at': datetime.datetime(2023, 1, 1, 0, 1), 'content': '안장', 'rental_info': {'rental_date': datetime.datetime(2023, 1, 3, 17, 47, 34), 'rental_place': '동묘앞역 1번출구 뒤', 'return_date': datetime.datetime(2023, 1, 3, 19, 59, 33), 'return_place': '종각역 5번출구'}}
{'_id

In [25]:
collection = db['rental_info']

pipeline = [
    {
        "$lookup": {
            "from": "rental_shop_info",
            "localField": "rental_station_id",
            "foreignField": "station_id",
            "as": "station_info"
        }
    },
    {"$unwind": "$station_info"},
    {"$project": {
        "cycle_num": 1,
        "rental_date": 1,
        "rental_place": 1,
        "return_date": 1,
        "return_place": 1,
        "station_info.address1": 1,
        "station_info.latitude": 1,
        "station_info.longitude": 1
    }},
    {"$limit": 5}
]

result = list(collection.aggregate(pipeline))
for doc in result:
    print(doc)

{'_id': ObjectId('665d6d02feb19af166ac3194'), 'cycle_num': 'SPB-39194', 'rental_date': datetime.datetime(2023, 1, 1, 0, 2, 13), 'rental_place': '번동사거리', 'return_date': datetime.datetime(2023, 1, 1, 0, 2, 43), 'return_place': '번동사거리', 'station_info': {'address1': '서울특별시 강북구 번동 430-76', 'latitude': 37.635391, 'longitude': 127.034554}}
{'_id': ObjectId('665d6d02feb19af166ac3195'), 'cycle_num': 'SPB-43457', 'rental_date': datetime.datetime(2023, 1, 1, 0, 1, 20), 'rental_place': '노원역1번출구', 'return_date': datetime.datetime(2023, 1, 1, 0, 3, 28), 'return_place': '상계초교 입구 교차로 명주빌딩 앞', 'station_info': {'address1': '서울특별시 노원구 상계동 335-4', 'latitude': 37.6562, 'longitude': 127.063622}}
{'_id': ObjectId('665d6d02feb19af166ac3196'), 'cycle_num': 'SPB-44383', 'rental_date': datetime.datetime(2023, 1, 1, 0, 4, 1), 'rental_place': '석촌호수 아뜰리에', 'return_date': datetime.datetime(2023, 1, 1, 0, 4, 21), 'return_place': '석촌호수 아뜰리에', 'station_info': {'address1': '서울특별시 송파구 삼학사로 136', 'latitude': 37.506748, 'l

In [26]:
collection = db['rental_info']

pipeline = [
    {"$group": {"_id": "$user.usr_cls_cd", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}}
]

result = list(collection.aggregate(pipeline))
for doc in result:
    print(doc)

{'_id': '내국인', 'count': 45899749}
{'_id': '비회원', 'count': 419404}
{'_id': '외국인', 'count': 69080}


In [31]:
import logging
# Logging
logging.info("Starting aggregation for broken_history and rental_info correlation")

pipeline = [
    {
        "$lookup": {
            "from": "rental_info",
            "localField": "cycle_num",
            "foreignField": "cycle_num",
            "as": "rental_info"
        }
    },
    {"$unwind": "$rental_info"},
    {
        "$group": {
            "_id": "$cycle_num",
            "total_rentals": {"$sum": 1},
            "total_broken": {"$sum": 1}
        }
    },
    {"$sort": {"total_broken": -1, "total_rentals": -1}},
    {"$limit": 10}
]

result = list(db['broken_history'].aggregate(pipeline))
for doc in result:
    logging.info(doc)

In [34]:
import logging
import pandas as pd
import matplotlib.pyplot as plt

# 인덱스 추가
db['broken_history'].create_index([('cycle_num', 1)])
db['rental_info'].create_index([('cycle_num', 1)])

# Logging
logging.info("Starting aggregation for broken_history and rental_info correlation")

pipeline = [
    {
        "$lookup": {
            "from": "rental_info",
            "localField": "cycle_num",
            "foreignField": "cycle_num",
            "as": "rental_info"
        }
    },
    {"$unwind": "$rental_info"},
    {
        "$group": {
            "_id": "$cycle_num",
            "total_rentals": {"$sum": 1},
            "total_broken": {"$sum": 1}
        }
    },
    {"$sort": {"total_broken": -1, "total_rentals": -1}},
    {"$limit": 10}
]

result = list(db['broken_history'].aggregate(pipeline))

# 데이터 프레임으로 변환
df = pd.DataFrame(result)

# 그래프로 가시화
plt.figure(figsize=(10, 6))
plt.scatter(df['_id'], df['total_broken'], label='Total Broken')
plt.scatter(df['_id'], df['total_rentals'], label='Total Rentals')
plt.xlabel('Cycle Number')
plt.ylabel('Count')
plt.title('Correlation between Broken Frequency and Rental Frequency')
plt.legend()
plt.show()

KeyboardInterrupt: 